In [ ]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip -q install accelerate>=0.12.0
# !pip install datasets

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 68.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd drive/MyDrive/en2sparql

Mounted at /content/drive
/content/drive/MyDrive/en2sparql


In [ ]:
import torch
import json
from transformers import pipeline, AutoTokenizer
# from datasets import load_dataset
import random


dolly = pipeline(model="databricks/dolly-v2-3b", torch_dtype=torch.bfloat16, trust_remote_code=True, device_map="auto")


def divide_chunks(l_, n_):
    for i_ in range(0, len(l_), n_):
        yield l_[i_:i_ + n_]


def clean(st):
    st = st.replace("\n", " ")
    st = st.replace("?", " ?")
    st = st.replace("{", " { ")
    st = st.replace("}", " } ")
    st = st.replace("\\'", "'")

    while "  " in st:
        st = st.replace("  ", " ")
    return st


def load_json(file__name):
    data_file = open(file__name, "r", encoding='utf-8')
    file_data = json.loads(data_file.read())
    data_file.close()
    return file_data


def save_json(filename,data):
    with open(filename, "w", encoding="utf-8") as json_file:
        print(json.dumps(data), file=json_file)


def main(batch=50):
  data = load_json("./dolly/test_3_T02_diversity_dolly_results.json.json")
  query_list = data.get("questions")
  suggestions = data.get("suggestions")
  sparql = data.get("sparql")
  gs = data.get("generated_sparql")
  query_list = query_list[len(gs):]

  n = batch

  q_list = list(divide_chunks(query_list, n))

  i = 0
  for group in q_list:
      print(str(i) + "%", end="  ")
      i += 1/len(q_list)*100

      res = dolly(group)
      print(res)
      gst = [x[0]["generated_text"] for x in res]

      for ii, l in enumerate(gst):
          for iii in range(10):
              if "SELECT" not in l:
                  print(iii,ii)
                  res = dolly(group[ii])
                  gst[ii] = res[0]["generated_text"]
                  l = gst[ii]
              else:
                  break
      gs += gst

      result = {"questions": query_list, "sparql": sparql, "generated_sparql": gs, "suggestions": suggestions}
      save_json("./dolly/test_3_T02_diversity_dolly_results.json", result)
      # break

main()

A new version of the following files was downloaded from https://huggingface.co/databricks/dolly-v2-3b:
- instruct_pipeline.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading the tokenizer from the `special_tokens_map.json` and the `added_tokens.json` will be removed in `transformers 5`,  it is kept for forward compatibility, but it is recommended to update your `tokenizer_config.json` by uploading it again. You will see the new `added_tokens_decoder` attribute that will store the relevant information.


0%  